# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request
import urllib

In [3]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = urllib.request.urlopen(wikipedia_url).read()
soup = BeautifulSoup(html, "html.parser")

### Steps
* Read the table in Wikipedia
* Go through one table row at a time
* Add each table row to a dictionary, which can then be used to create the DataFrame

In [4]:
results = []
table = soup.find("table",{"class":"wikitable sortable"})
trs = table.findAll('tr')
trs = [tr for tr in trs if len(tr.find_all('th'))<1 ]
for tr in trs:
    tds = tr.find_all('td')
    #     PostalCode, Borough, and Neighborhood
    d = {
        'PostalCode': str(tds[0].text.strip()),
        'Borough': str(tds[1].text.strip()),
        'Neighborhood': str(tds[2].text.strip())
    }
    results.append(d)


### Create the DataFrame

In [5]:
df = pd.DataFrame(results)
df.head()

,Borough,Neighborhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


### Ignore cells with a borough that is Not assigned.

In [6]:
df = df[df.Borough != 'Not assigned']

### Combine Neighbourhoods belonging to same PostalCode and Borough

In [7]:
df = df.groupby(['PostalCode', 'Borough']).agg(','.join).reset_index()
df[df.PostalCode == 'M9V']

,PostalCode,Borough,Neighborhood
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
df['Neighborhood'] = np.where(df.Neighborhood == 'Not assigned', df.Borough, df.Neighborhood)

In [9]:
df[df.PostalCode == 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


### Total Number of Rows

In [10]:
df.shape

(103, 3)

## *** Second Question ***

#### I created a separate notebook as the instructions were a bit ambiguous

In [11]:
lat_lng_file = 'https://cocl.us/Geospatial_data'
lat_lng = pd.read_csv(lat_lng_file)
lat_lng.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


### Merge the Lat Lng Dataframe with the Locations DF

In [12]:
df = df.merge(lat_lng, left_on='PostalCode', right_on = 'Postal Code', how='left')

In [13]:
df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


#### We can drop one extra Postal Code column

In [14]:
df.drop('Postal Code', axis=1, inplace=True)

### Make sure we have not increased the number of rows. It should still be 103 rows and 5 columns

In [15]:
df.shape

(103, 5)

In [16]:
df.head(3)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711


# **** Third Question ****

### Explore & Cluster neighborhoods in Toronto

In [23]:
print("The boroughs in Toronto are : %s"%df[df.Borough.str.contains('Toronto')].Borough.unique().tolist())

The boroughs in Toronto are : ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


In [24]:
toronto_df = df[df.Borough.str.contains('Toronto')]
toronto_df.shape

(38, 5)

In [45]:
import folium
import requests
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize

#### Get Toronto Coordinates

In [27]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [493]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [494]:
# @hidden cell
CLIENT_ID = 'DFELEIN3LFIR3CVHUFUOCAP1CT4THHW3Y0YUKIGPKBOIIJQZ' 
CLIENT_SECRET = 'LV0LAP0DY0MYA4QGRCF5EFD1B2DGTDPE10DFWZESBBP3KKE1' 
VERSION = '20180605' # Foursquare API version


In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### For each Postal Code, find the top 100 venues using FourSquare APIs, restricted to a radius of 1 Km

In [175]:
postcodes = toronto_df.PostalCode.unique().tolist()
print("We have %s postal codes to search in Toronto"%len(postcodes))

We have 38 postal codes to search in Toronto


In [176]:
radius = 1000
limit = 100
venues_list=[]
for postcode in postcodes:
    lat = toronto_df[toronto_df.PostalCode == postcode].iloc[0]['Latitude']
    lng = toronto_df[toronto_df.PostalCode == postcode].iloc[0]['Longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        limit)

    print("Seaching for postal code = %s"%postcode)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    venues_list.append([(
            postcode, 
            toronto_df[toronto_df.PostalCode == postcode].iloc[0]['Borough'],
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])
    

Seaching for postal code = M4E
Seaching for postal code = M4K
Seaching for postal code = M4L
Seaching for postal code = M4M
Seaching for postal code = M4N
Seaching for postal code = M4P
Seaching for postal code = M4R
Seaching for postal code = M4S
Seaching for postal code = M4T
Seaching for postal code = M4V
Seaching for postal code = M4W
Seaching for postal code = M4X
Seaching for postal code = M4Y
Seaching for postal code = M5A
Seaching for postal code = M5B
Seaching for postal code = M5C
Seaching for postal code = M5E
Seaching for postal code = M5G
Seaching for postal code = M5H
Seaching for postal code = M5J
Seaching for postal code = M5K
Seaching for postal code = M5L
Seaching for postal code = M5N
Seaching for postal code = M5P
Seaching for postal code = M5R
Seaching for postal code = M5S
Seaching for postal code = M5T
Seaching for postal code = M5V
Seaching for postal code = M5W
Seaching for postal code = M5X
Seaching for postal code = M6G
Seaching for postal code = M6H
Seaching

#### Build a Dataframe containing all the venues

In [495]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['PostalCode', 'Borough',
              'PostalCode Latitude', 
              'PostalCode Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']


In [496]:
print("Found %s Categories"%len(nearby_venues['Venue Category'].unique().tolist()))

Found 276 Categories


### That's a lot of categories !!!
### Let's look at the Top 15 Categories....and see if we can combine some of them.

In [497]:
nearby_venues.groupby(['Venue Category']).count().sort_values('PostalCode', ascending=False).reset_index().head(15)

,Venue Category,PostalCode,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude
0,Coffee Shop,234,234,234,234,234,234,234
1,Café,170,170,170,170,170,170,170
2,Italian Restaurant,91,91,91,91,91,91,91
3,Restaurant,78,78,78,78,78,78,78
4,Bakery,77,77,77,77,77,77,77
5,Park,75,75,75,75,75,75,75
6,Pizza Place,62,62,62,62,62,62,62
7,Bar,62,62,62,62,62,62,62
8,Hotel,62,62,62,62,62,62,62
9,Sushi Restaurant,56,56,56,56,56,56,56


#### Seems like we have plenty of Restaurants (Indian, Chineese, Turkish, Italian, Greek, Pizza, Steakhouse, etc.) . Let's group them under a generic "Restaurant" category

In [498]:
mask = nearby_venues['Venue Category'].str.contains('Restaurant')
nearby_venues.loc[mask, 'Venue Category'] = 'Restaurant'
nearby_venues['Venue Category'] = np.where(nearby_venues['Venue Category'].isin(['Diner', 'Burger Joint', 'Burrito Place', 'Steakhouse', 'Pizza Place']), 'Restaurant', nearby_venues['Venue Category'])

#### Merge Coffee Shops, Cafe, Bakery, Breakfast spot, Tea Room into a single category called "Cafe"

In [499]:
nearby_venues['Venue Category'] = np.where(nearby_venues['Venue Category'].isin(['Café', 'Sandwich Place', 'Tea Room', 'Coffee Shop', 'Bakery', 'Breakfast Spot', 'Ice Cream Shop']), 'Café', nearby_venues['Venue Category'])

#### Merge Pub, Bar, GastroPub into a single category

In [500]:
nearby_venues['Venue Category'] = np.where(nearby_venues['Venue Category'].isin(['Pub', 'Gastropub', 'Bar', 'Beer Bar', 'Cocktail Bar', 'Brewery']), 'Bar', nearby_venues['Venue Category'])

In [501]:
print("Found %s Categories after grouping restaurants"%len(nearby_venues['Venue Category'].unique().tolist()))

Found 207 Categories after grouping restaurants


In [502]:
nearby_venues.groupby(['Venue Category']).count().sort_values('PostalCode', ascending=False).reset_index().head(10)

,Venue Category,PostalCode,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude
0,Restaurant,944,944,944,944,944,944,944
1,Café,612,612,612,612,612,612,612
2,Bar,229,229,229,229,229,229,229
3,Park,75,75,75,75,75,75,75
4,Hotel,62,62,62,62,62,62,62
5,Gym,50,50,50,50,50,50,50
6,Grocery Store,32,32,32,32,32,32,32
7,Theater,32,32,32,32,32,32,32
8,Bookstore,30,30,30,30,30,30,30
9,Deli / Bodega,24,24,24,24,24,24,24


In [503]:
nearby_venues.shape

(3089, 8)

### Analyse Each Postal Code Neighborhood
### Looks like Toronto is predominantly dominated the first 3 categories - Restaurants, Bars and Cafes

In [505]:
# one hot encoding
toronto_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['PostalCode'] = nearby_venues['PostalCode'] 
toronto_onehot.head()

,Accessories Store,Airport,Airport Lounge,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,University,Video Game Store,Video Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo,PostalCode
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4E
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4E
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4E
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4E
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M4E


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [506]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Airport,Airport Lounge,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Train Station,University,Video Game Store,Video Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M4E,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,0.000000,0.0
1,M4K,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.020000,0.0
2,M4L,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.012821,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012821,0.012821,0.0
3,M4M,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.010000,0.0
4,M4N,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


### For each postal code we find the Top 3 categories

In [507]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [508]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Restaurant,Café,Bar,Beach,Park
1,M4K,Restaurant,Café,Bar,Yoga Studio,Bank
2,M4L,Restaurant,Café,Bar,Beach,Gym
3,M4M,Restaurant,Café,Bar,Gym / Fitness Center,Park
4,M4N,Café,Bookstore,College Quad,Gym / Fitness Center,College Gym


### Start Clustering Neighborhoods into 3 clusters

In [509]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn import metrics

In [510]:
kclusters = 3
print("Number of Clusters = %s"%kclusters)

Number of Clusters = 3


In [511]:
toronto_clusters = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(toronto_clusters)


In [512]:
# ADD CLUSTERED LABELS AND CREATE A NEW MERGED DF
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_df_with_clusters = toronto_df
toronto_df_with_clusters = toronto_df_with_clusters.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')
toronto_df_with_clusters.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Restaurant,Café,Bar,Beach,Park
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Restaurant,Café,Bar,Yoga Studio,Bank
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Restaurant,Café,Bar,Beach,Gym
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Restaurant,Café,Bar,Gym / Fitness Center,Park
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Café,Bookstore,College Quad,Gym / Fitness Center,College Gym


### Visualize the Clusters

In [513]:
# create map
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['blue', 'red', 'green', 'yellow', 'orange', 'purple', 'grey']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df_with_clusters['Latitude'], toronto_df_with_clusters['Longitude'], toronto_df_with_clusters['PostalCode'], toronto_df_with_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyse Clusters

## Cluster 1

In [514]:
toronto_df_with_clusters[toronto_df_with_clusters['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Restaurant,Café,Bar,Beach,Park
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Restaurant,Café,Bar,Gym / Fitness Center,Park
53,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Restaurant,Café,Bar,Theater,Park
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Restaurant,Café,Bar,Hotel,Cosmetics Shop
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Restaurant,Café,Bar,Hotel,Park
59,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Café,Restaurant,Bar,Hotel,Aquarium
61,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0,Restaurant,Café,Bar,Hotel,Concert Hall
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Restaurant,Café,Bank,Pharmacy,Dance Studio
64,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,0,Restaurant,Café,Gym / Fitness Center,Skating Rink,Liquor Store
69,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Restaurant,Café,Bar,Hotel,Gym


#### Cluster 1 looks like its locations with Restaurants and Cafes as the 1st and 2nd Most common venues

## Cluster 2

In [515]:
toronto_df_with_clusters[toronto_df_with_clusters['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Restaurant,Café,Bar,Yoga Studio,Bank
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Restaurant,Café,Bar,Beach,Gym
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Restaurant,Café,Dessert Shop,Bar,Gym
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Restaurant,Café,Sporting Goods Shop,Skating Rink,Gym / Fitness Center
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Restaurant,Café,Bar,Gym,Dessert Shop
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Restaurant,Café,Gym,Grocery Store,Park
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1,Restaurant,Café,Park,Bar,Liquor Store
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,1,Restaurant,Café,Park,Bar,Theater
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Restaurant,Café,Bar,Gay Bar,Hotel
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Restaurant,Café,Bar,Clothing Store,Bookstore


#### Cluster 2 looks like its locations with Restaurants as the first common venue and Cafes as the 2nd Most common venues

## Cluster 3

In [516]:
toronto_df_with_clusters[toronto_df_with_clusters['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Café,Bookstore,College Quad,Gym / Fitness Center,College Gym
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Café,Grocery Store,Park,Restaurant,Bank
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,2,Café,Harbor / Marina,Park,Garden,Sculpture Garden


#### Cluster 3 looks like areas that are mixed with Cafes, Parks, and other facilities in Toronto.....Could these be more residential?